In [9]:
import pandas as pd
import os
import sqlite3
import time
import logging

# Set up logging
logging.basicConfig(
    filename="logs/get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)


def ingest_db(df, table_name, engine):
    '''Ingest a DataFrame into a database table'''
    df.to_sql(table_name, con=engine, if_exists='replace', index=False,chunksize=1000)
    logging.info(f'Table {table_name} ingested successfully.')

def create_vendor_summary(conn):
    '''This function will merge the different tables to get the overall vendor summary and adding new  columns in the resultant data'''
    vendor_sales_summary = pd.read_sql_query("""with fs as (
    select VendorNumber,
    sum(Freight) as FreightCost
    from vendor_invoice
    group by VendorNumber
),ps as (
    Select
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.Description,
    p.PurchasePrice,
    pp.Price as ActualPrice,
    pp.Volume,
    sum(p.Quantity) as TotalPurchaseQuantity,
    sum(p.Dollars) as TotalPurchaseDollars
    from purchases p
    join purchase_prices pp
        on p.Brand = pp.Brand
    where p.PurchasePrice > 0
    GROUP BY p.VendorNumber,p.VendorName,p.Brand,p.Description,p.PurchasePrice,pp.Volume
),
ss as (
    select
    VendorNo,
    Brand,
    sum(SalesQuantity) as TotalSalesQuantity,
    sum(SalesDollars) as TotalSalesDollars,
    sum(SalesPrice) as TotalSalesPrice,
    sum(ExciseTax) as TotalExciseTax
    from sales
    GROUP BY VendorNo,Brand
)
    select
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
    from ps
    left join ss
        ON ps.VendorNumber = ss.VendorNo
        AND ps.Brand = ss.Brand
    left join fs
        ON ps.VendorNumber = fs.VendorNumber
    order by ps.TotalPurchaseDollars desc""",conn)
    return vendor_sales_summary

def clean_data(df):
    '''this function will clean the data'''
    df['Volume'] = df['Volume'].astype('float')
    df.fillna(0,inplace=True)
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # Creating new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = (df['GrossProfit'] / df['TotalSalesDollars'])*100
    df['StockTurnover'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    df['SalesPurchaseRatio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']
    return df

if __name__ == '__main__':
    # creating database connection
    conn = sqlite3.connect('inventory.db')
    
    logging.info('Creating vendor summary table.........')
    df = create_vendor_summary(conn)
    logging.info(df.head())

    logging.info('Cleaning Data..........')
    clean_df = clean_data(df)
    logging.info(clean_df.head())

    logging.info('Ingesting Data.........')
    ingest_db(clean_df,'vendor_sales_summary',conn)
    logging.info('ingested successfully Compleated :)')